In [ ]:
%pip install pandas numpy matplotlib tensorflow keras

In [ ]:
%pip install xlrd

In [22]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import ticker as ticker

In [23]:
train_data = pd.read_excel(r"./data/Data_Train.xlsx", sheet_name="Sheet1")

In [24]:
train_data = train_data.dropna(how='any',axis=0)
train_data = train_data.to_numpy()

[['Honda Jazz V' 'Chennai' 2011 ... 5.0 '8.61 Lakh' 4.5]
 ['Toyota Innova Crysta 2.8 GX AT 8S' 'Mumbai' 2016 ... 8.0 '21 Lakh'
  17.5]
 ['Maruti Ciaz Zeta' 'Kochi' 2018 ... 5.0 '10.65 Lakh' 9.95]
 ...
 ['Maruti Vitara Brezza VDi' 'Pune' 2016 ... 5.0 '9.93 Lakh' 7.43]
 ['Honda Brio 1.2 VX MT' 'Delhi' 2013 ... 5.0 '6.63 Lakh' 3.2]
 ['Maruti Swift VDI' 'Delhi' 2014 ... 5.0 '7.88 Lakh' 4.75]]


In [25]:
# assume `Price` in Lakh
data_params = ['Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats', 'New_Price', 'Price']

In [26]:
def one_hot_encode(data:np.ndarray, n:int, categories:list = None) -> np.ndarray:
    """
    Convert the given input into one-hot-encoding.
    Use the given categories, if exist then convert the data,
    else convert indices to one-hot
    """

    try:
        assert categories is None
        data = data.astype(int)
    except (ValueError, AssertionError):
        data = np.vectorize(categories.index)(data)
    finally:
        targets = np.array(data).reshape(-1)
    
    return np.eye(n)[targets][0]

In [27]:
one_encode_params = ["Name", "Location", "Year", "Fuel_Type", "Transmission", "Owner_Type"]

def encode_params(data, params):
    ix = map(data_params.index, params) 
    encoded_params = {}

    for i in ix:
        m = data_params[i]
        encoded_params[m] = [np.unique(data[:,i].astype(str), axis=0).tolist(), None]
        encoded_params[m][1] = list(map(lambda x: one_hot_encode(x, len(encoded_params[m][0]), encoded_params[m][0]), data[:,i].astype(str)))
    
    return encoded_params

In [28]:
categories = encode_params(train_data, one_encode_params)

In [29]:
def convert_prices(price):
    p = price.split(" ")
    price = float(p[0])
    cr = p[1] == "Cr"

    return price if not cr else 100*price

train_data[:,11] = np.vectorize(convert_prices)(train_data[:,11]).astype(float)

In [34]:
train_data[:,7] = np.vectorize(lambda i: float(i.split(" ")[0]))(train_data[:,7]).astype(float)

In [39]:
train_data[:,8] = np.vectorize(lambda i: float(i.split(" ")[0]))(train_data[:,8]).astype(float)
train_data[:,9] = np.vectorize(lambda i: float(i.split(" ")[0]))(train_data[:,9]).astype(float)

In [41]:
train_data[:2]

array([['Honda Jazz V', 'Chennai', 2011, 46000, 'Petrol', 'Manual',
        'First', 18.2, 1199.0, 88.7, 5.0, 8.61, 4.5],
       ['Toyota Innova Crysta 2.8 GX AT 8S', 'Mumbai', 2016, 36000,
        'Diesel', 'Automatic', 'First', 11.36, 2755.0, 171.5, 8.0, 21.0,
        17.5]], dtype=object)